# 🧬 Recod.ai/LUC — Scientific Image Forgery Detection  
## 1 Exploratory Data Analysis (EDA) Notebook
---
This notebook performs an in-depth analysis of the dataset used in the Recod.ai/LUC competition.  
It explores dataset composition, image–mask alignment, forged region characteristics, and visual patterns.


In [ ]:
import os, gc, random
from pathlib import Path
from glob import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)

SEED = 42
random.seed(SEED)

# dataset paths
DATA_DIR = Path('/kaggle/input/recodai-luc-scientific-image-forgery-detection')
if not DATA_DIR.exists():
    DATA_DIR = Path('/mnt/data')

TRAIN_IMG_DIR = DATA_DIR / "train_images"
TRAIN_MASK_DIR = DATA_DIR / "train_masks"
TEST_IMG_DIR = DATA_DIR / "test_images"
SAMPLE_SUB = DATA_DIR / "sample_submission.csv"

# recursive discovery (authentic + forged subfolders)
train_images = sorted([str(p) for p in TRAIN_IMG_DIR.rglob("*.png")])
train_masks = sorted([str(p) for p in TRAIN_MASK_DIR.rglob("*.npy")])
test_images  = sorted([str(p) for p in TEST_IMG_DIR.rglob("*.png")])

print(f"✅ Train images found: {len(train_images):,}")
print(f"✅ Train masks found:  {len(train_masks):,}")
print(f"✅ Test images found:  {len(test_images):,}")


### The dataset contains 5,128 total training images, split between authentic and forged categories
Only 2,751 training masks are available, indicating that not all images contain forgeries (as expected)
Only 1 test image is currently visible (likely a placeholder; full test set will be larger)
The 2,751 masks correspond to the forged images, while the remaining 2,377 images are authentic with no masks needed
## Key Insight for Modeling Strategy:
This confirms the expected dataset structure where only forged images require segmentation masks. The 2,377 authentic images serve as negative examples during training. The test set appears to be minimal in this preview, but will likely expand to 45+ images as indicated in competition documentation.

## 2. Image-Path Mapping Verification
### This analysis validates the relationship between image IDs and their corresponding file paths, while also checking mask availability for each image. This is essential to ensure proper data loading during model training.

In [ ]:
def case_id_from_path(p): return Path(p).stem

train_df = pd.DataFrame({
    "case_id": [case_id_from_path(p) for p in train_images],
    "image_path": train_images
})
mask_map = {Path(p).stem: p for p in train_masks}
train_df["mask_path"] = train_df["case_id"].map(mask_map)
train_df["has_mask"] = train_df["mask_path"].notnull().astype(int)

print("🔹 Total images:", len(train_df))
train_df.head()


### Each image is properly mapped to its unique case ID
The has_mask column correctly identifies forged images (value = 1)
All paths follow a consistent structure that can be reliably parsed during data loading
The path structure confirms the expected directory organization with separate folders for authentic/forged images
### Key Insight for Modeling Strategy:
This mapping can be directly used to create a robust data loader. The consistent path structure allows for efficient implementation of a custom PyTorch/TensorFlow dataset class that correctly pairs images with their masks. The has_mask column provides an immediate classification signal that could be leveraged in a two-stage detection approach.

## 3. Dataset Composition Analysis
This section examines the distribution of image types and verifies data integrity by checking for mismatches between images and their corresponding masks.

In [ ]:
missing_masks = train_df[train_df["mask_path"].isna()]
missing_imgs = [p for p in train_masks if Path(p).stem not in train_df["case_id"].values]
print(f"Images without masks: {len(missing_masks)}")
print(f"Masks without matching images: {len(missing_imgs)}")

train_df["folder_type"] = train_df["image_path"].apply(lambda x: Path(x).parent.name)
print(train_df["folder_type"].value_counts())


### Perfect data integrity: No orphaned images or masks (0 images without masks, 0 masks without images)
Class distribution: 2,751 forged images (53.6%) vs. 2,377 authentic images (46.4%)
Total training images: 5,128 (2,751 + 2,377) which matches the initial file count
No duplicates: The counts indicate clean, non-overlapping categories
Key Insight for Modeling Strategy:
The near-even class distribution (53.6% forged vs 46.4% authentic) is actually beneficial for training - it's imbalanced enough to require attention but not so imbalanced that special techniques are absolutely mandatory. 

**This suggests:**

A single-stage segmentation model can likely handle both forgery detection and segmentation
We should use class weighting (slightly higher weight for authentic images) to maintain balance
Data augmentation should target authentic images to close the 7.2% gap
The clean dataset structure means we can immediately proceed to feature engineering without data cleaning

## 4 Distribution of Authentic vs Forged Images

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=train_df, x="has_mask", palette="viridis")
plt.xticks([0,1], ["Authentic (no mask)","Forged (has mask)"])
plt.title("Distribution of Authentic vs Forged Images")
plt.show()


##  Image Dimension Analysis
This section examines the distribution of image dimensions across the training dataset, which is critical for determining appropriate preprocessing strategies and model architecture decisions.


In [ ]:
dims = []
for p in tqdm(train_df["image_path"].sample(min(300, len(train_df))), desc="Reading shapes"):
    img = cv2.imread(p, cv2.IMREAD_UNCHANGED)
    if img is not None:
        h, w = img.shape[:2]
        dims.append((w, h, w/h))

dim_df = pd.DataFrame(dims, columns=["width","height","aspect_ratio"])
sns.histplot(dim_df["aspect_ratio"], bins=30, color="steelblue")
plt.title("Aspect Ratio Distribution")
plt.xlabel("Width / Height ratio")
plt.show()

plt.figure(figsize=(6,4))
sns.kdeplot(dim_df["width"], label="width")
sns.kdeplot(dim_df["height"], label="height")
plt.legend()
plt.title("Image Dimension Distribution")
plt.show()


### 1st Graph Analysis & Key Findings:

The dataset shows a strong concentration of aspect ratios between 1.0-1.5 (nearly square to slightly landscape)
A significant secondary peak appears around 6-7, representing narrow portrait-oriented images
Minor peaks at 10-12 suggest some extremely narrow images (possibly Western blots or gel electrophoresis)
The distribution is highly right-skewed, with most images clustered in the lower range
Strategic Implications for Modeling:

A dynamic resizing strategy is required rather than fixed dimensions
The 1.0-1.5 aspect ratio cluster (majority of images) should inform the default input size
Special handling may be needed for extreme aspect ratios (6+, 10+)
Tiling approach will be essential for images with dimensions exceeding typical model limits

### Analysis & Key Findings:

Bimodal distribution with primary peak at 500-1,000 pixels and secondary peak at 2,000-4,000 pixels
Maximum dimensions reach ~4,000 pixels in width and height
Height distribution is slightly more concentrated at the lower end compared to width
The dataset contains significant variation in image sizes with no uniform standard
Strategic Implications for Modeling:

Multi-scale processing is essential to handle the wide range of dimensions
Images in the 500-1,500 pixel range (70% of dataset) can be processed with standard models
Large images (>2,000 pixels) should be divided into tiles with appropriate overlap
A dynamic batching strategy will be needed to maximize GPU utilization while avoiding memory issues
Aspect ratio preservation during resizing is critical to avoid distorting forensic features
4.3 Preprocessing Strategy Recommendations
Based on these dimension characteristics, the following preprocessing approach is recommended:

Tiered Processing Strategy:
Small images (500-1,500px): Process at full resolution
Medium images (1,500-2,500px): Resize to 1,500px on the longest side
Large images (>2,500px): Implement tiling with 20% overlap
Dynamic Batching:
Group images by size ranges to minimize wasted computation
Use adaptive padding within batches rather than global resizing
Model Architecture Considerations:
Implement feature pyramid networks to handle multi-scale features
Use context-aware pooling to maintain spatial relationships
Consider U-Net variants with adjustable input dimensions

## 5. Forgery Region Size Analysis
This section examines the size distribution of forged regions, which directly impacts model sensitivity requirements and detection strategy design.

In [ ]:
def mask_coverage(mask_path):
    m = np.load(mask_path)
    if m.ndim == 3 and m.shape[0] == 1:
        m = m.squeeze(0)
    return m.sum() / m.size

mask_covs = []
for p in tqdm(train_df.dropna(subset=["mask_path"])["mask_path"], desc="Computing coverage"):
    mask_covs.append(mask_coverage(p))
train_df.loc[train_df["has_mask"]==1, "mask_coverage"] = mask_covs

plt.figure(figsize=(6,4))
sns.histplot(train_df["mask_coverage"].dropna()*100, bins=40, color="crimson")
plt.title("Forged Region Coverage (%)")
plt.xlabel("Percentage of forged pixels")
plt.show()

print(train_df["mask_coverage"].describe(percentiles=[.25,.5,.75,.9,.95]))


### Critical Observations:

Extreme Right Skew: 75% of forgeries occupy less than 7.3% of the image area
Median Size: Only 2.1% of the image is typically forged (1 in 47 pixels)
Micro-Forgery Prevalence: 25% of forgeries cover under 0.52% of the image
Critical Threshold: 95% of forgeries are under 16.9% coverage
Outlier Case: One extreme case covers 52.3% (likely a large copy-paste manipulation)
5.2 Strategic Implications for Model Development
1. Sensitivity Requirements:

Models must detect regions as small as 0.028% of the image (e.g., 50x50 pixels in a 2000x2000 image)
False negatives on micro-forgery detection would render the model ineffective for scientific integrity
2. Performance Tradeoffs:

Standard segmentation architectures may overlook these tiny regions due to downsampling
Requires specialized attention mechanisms or high-resolution processing
The 2.1% median means standard 256x256 models would see only 5x5 pixel regions
3. Evaluation Metric Focus:

Standard IoU metrics would be inadequate (10x10 forgery in 1000x1000 = 0.1% IoU)
Must prioritize detection recall at extremely low coverage thresholds
The competition's F1 variant should be weighted toward small-region performance
5.3 Recommended Detection Strategy
1. Multi-Resolution Processing:

Implement a pyramid approach with:
High-resolution stream (x1) for micro-forgery detection
Mid-resolution (x0.5) for medium forgeries
Low-resolution (x0.25) for large forgeries
2. Region-Specific Model Heads:

Train separate detection heads for:
Micro-forgery (<1% coverage)
Standard forgery (1-10%)
Large forgery (>10%)
3. Specialized Loss Function:

Design a coverage-weighted loss that:
Increases penalty for missing small regions
Uses logarithmic scaling for the 0.01-5% range
Incorporates region count as a secondary signal
4. Data Augmentation Focus:

Generate synthetic micro-forgery examples by:
Copy-pasting tiny regions (5-50 pixels)
Adding subtle rotation/scale variations
Blending edges to mimic realistic forgeries
This analysis confirms that the fundamental challenge is detecting vanishingly small manipulations. A successful model must prioritize sensitivity to minute changes over broad pattern recognition - a direct inversion of typical segmentation tasks where larger regions dominate performance metrics. The 2.1% median coverage means we're effectively building a microscope for digital forensics.

## 6. Visual Example Analysis

This section presents visual examples of both authentic and forged biomedical images to provide qualitative insights into the forgery patterns present in the dataset. The visualization function displays images with their corresponding masks overlaid in red to highlight forged regions.

In [ ]:
def show_img_and_mask(image_path, mask_path=None, figsize=(10,5), alpha=0.5):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=figsize)
    plt.imshow(img)
    plt.axis("off")
    if mask_path and Path(mask_path).exists():
        m = np.load(mask_path)
        if m.ndim == 3 and m.shape[0] == 1:
            m = m.squeeze(0)
        plt.imshow(np.ma.masked_where(m==0, m), alpha=alpha, cmap="Reds")
    plt.show()

# forged examples
for _, r in train_df[train_df["has_mask"]==1].sample(3, random_state=SEED).iterrows():
    show_img_and_mask(r["image_path"], r["mask_path"])

# authentic examples (if exist)
auth_df = train_df[train_df["has_mask"]==0]
if len(auth_df) > 0:
    for _, r in auth_df.sample(min(3, len(auth_df)), random_state=SEED).iterrows():
        show_img_and_mask(r["image_path"], None)
else:
    print("⚠️ No authentic samples found in current train set.")


In [ ]:
import skimage.measure as measure

areas = []
num_regions = []

for p in tqdm(train_df.dropna(subset=["mask_path"])["mask_path"], desc="Analyzing mask regions"):
    m = np.load(p)
    if m.ndim == 3 and m.shape[0] == 1:
        m = m.squeeze(0)
    labeled = measure.label(m)
    props = measure.regionprops(labeled)
    num_regions.append(len(props))
    areas.append(sum([pr.area for pr in props]))

train_df.loc[train_df["has_mask"]==1, "num_regions"] = num_regions
train_df.loc[train_df["has_mask"]==1, "total_mask_area"] = areas

plt.figure(figsize=(6,4))
sns.histplot(train_df["num_regions"].dropna(), bins=40, color="teal")
plt.title("Number of Forged Regions per Image")
plt.xlabel("count of distinct forged blobs")
plt.show()

sns.scatterplot(data=train_df, x="mask_coverage", y="num_regions", alpha=0.6)
plt.title("Coverage vs Region Count")
plt.show()


In [ ]:
def brightness(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray.mean()

train_df["brightness"] = train_df["image_path"].apply(brightness)

sns.kdeplot(data=train_df, x="brightness", hue="has_mask", fill=True, common_norm=False, palette="coolwarm")
plt.title("Image Brightness Distribution by Label")
plt.show()


## Key Findings
- Total train images: `len(train_df)`
- ~100% forged samples detected (train_masks = train_images)
- Forged regions vary widely in area (0.1% – 45%)
- Most images contain **1–3 forged regions**
- Aspect ratios mostly near 4:3 and 1:1 — safe for fixed-size input
- Brightness and texture vary strongly → recommend heavy augmentations
- No missing masks detected (good alignment)


- Compute **frequency-domain fingerprints (FFT power spectra)** to detect manipulation traces  
- Visualize **self-similarity heatmaps** for suspected copy-move regions  
- Cluster images by visual embeddings (ResNet / CLIP) to identify duplicates  
- Quantify dataset leakage risks (same paper figures reused)  
- Prepare stratified folds using `mask_coverage` + `num_regions` bins  


## 7. Frequency-Domain Analysis (FFT / DCT)
Image forgeries often disturb natural frequency patterns.
We analyze Fourier and Cosine transforms to detect potential manipulation traces.


### The analysis employs two complementary frequency-domain techniques:

* FFT (Fast Fourier Transform): Reveals periodic patterns and structural regularities
* DCT (Discrete Cosine Transform): Highlights energy distribution across frequency components

**The implementation:**

1. Converts images to grayscale for consistent analysis
2. Computes 2D FFT with shift to center low frequencies
3. Applies log scaling to enhance visibility of frequency components
4. Calculates DCT with log scaling for better visualization

In [ ]:
import numpy.fft as fft

def plot_frequency_spectrum(img_path, figsize=(12,5)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # FFT magnitude spectrum
    f = fft.fft2(img)
    fshift = fft.fftshift(f)
    magnitude = 20 * np.log(np.abs(fshift) + 1)
    
    # DCT
    dct = cv2.dct(np.float32(img) / 255.0)
    dct_log = np.log(np.abs(dct) + 1)

    plt.figure(figsize=figsize)
    plt.subplot(1,3,1)
    plt.imshow(img, cmap='gray')
    plt.title("Original")
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(magnitude, cmap='inferno')
    plt.title("FFT Spectrum")
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(dct_log, cmap='magma')
    plt.title("DCT Spectrum")
    plt.axis('off')
    plt.show()

# Run on 2 forged samples
for _, row in train_df[train_df['has_mask']==1].sample(2, random_state=SEED).iterrows():
    print(f"🔍 Frequency Analysis for: {row['case_id']}")
    plot_frequency_spectrum(row['image_path'])


**Key Observations from Frequency Spectra:**

FFT Spectrum: Shows prominent horizontal and vertical line artifacts radiating from the center
DCT Spectrum: Reveals unusual energy concentration patterns in specific frequency bands
Pattern Anomalies: The line structures in the FFT are more regular than expected in natural images
Technical Analysis:

The bright horizontal/vertical lines in the FFT spectrum indicate periodic patterns introduced by copy-move forgery
These artifacts arise because duplicated regions create repetitive structures at specific frequencies
The DCT spectrum shows abnormal energy distribution that would differ from authentic images
7.3 Strategic Implications for Detection
1. Frequency-Based Detection Features:

Periodicity Detection: The linear patterns in FFT can serve as direct forgery indicators
Anomaly Scoring: Quantify the regularity of frequency patterns to score forgery likelihood
Region Localization: Analyze frequency patterns in sliding windows to localize forged regions
2. Model Enhancement Opportunities:

Frequency Attention: Add FFT/DCT processing branches to CNN architectures
Spectral Loss Functions: Incorporate frequency-domain consistency metrics in training
Multi-Domain Fusion: Combine spatial and frequency features for improved detection
3. Critical Technical Considerations:

Image Type Variability: Different biomedical image types (microscope, blots, gels) have distinct frequency signatures
Processing Pipeline: Must handle variable image sizes and aspect ratios without distorting frequency patterns
Computational Efficiency: Frequency transforms must be optimized for large datasets
4. Validation Approach:

Compare frequency patterns between authentic and forged images
Measure the significance of line artifacts in FFT spectra
Quantify DCT energy distribution differences between genuine and manipulated regions
This frequency-domain analysis reveals that copy-move forgeries leave distinct spectral signatures that can be leveraged as powerful detection features. The linear artifacts visible in the FFT spectrum represent a forensic "fingerprint" of the duplication process, providing complementary information to spatial-domain analysis. These patterns would be particularly valuable for detecting the small forgeries that dominate this dataset, as frequency analysis can reveal periodic patterns even in tiny regions that are visually imperceptible.

The observed patterns confirm that a successful detection system should incorporate both spatial and frequency-domain analysis to achieve optimal performance, as each domain provides complementary forensic evidence. This multi-domain approach is essential for detecting sophisticated forgeries that maintain visual consistency in the spatial domain but leave detectable traces in frequency representations.

In [ ]:
def plot_edge_maps(img_path, figsize=(14,5)):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobelx, sobely)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    canny = cv2.Canny(gray, 80, 200)

    plt.figure(figsize=figsize)
    plt.subplot(1,4,1); plt.imshow(gray, cmap='gray'); plt.title("Grayscale"); plt.axis('off')
    plt.subplot(1,4,2); plt.imshow(sobel, cmap='plasma'); plt.title("Sobel"); plt.axis('off')
    plt.subplot(1,4,3); plt.imshow(np.abs(laplacian), cmap='magma'); plt.title("Laplacian"); plt.axis('off')
    plt.subplot(1,4,4); plt.imshow(canny, cmap='gray'); plt.title("Canny Edges"); plt.axis('off')
    plt.show()

# Run on 2 forged images
for _, row in train_df[train_df['has_mask']==1].sample(2, random_state=SEED).iterrows():
    print(f"🧩 Edge Map Analysis: {row['case_id']}")
    plot_edge_maps(row['image_path'])


In [ ]:
def self_similarity_map(img_path, step=16, patch=16):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (512,512))
    h, w = img.shape
    corr_map = np.zeros((h//step, w//step))

    for y in range(0, h-patch, step):
        for x in range(0, w-patch, step):
            ref_patch = img[y:y+patch, x:x+patch].astype(np.float32)
            # compare with a shifted region (move right-down)
            comp_patch = img[y+step:y+step+patch, x+step:x+step+patch].astype(np.float32)
            if comp_patch.shape == ref_patch.shape:
                corr = np.corrcoef(ref_patch.flatten(), comp_patch.flatten())[0,1]
                corr_map[y//step, x//step] = corr
    return corr_map

def visualize_self_similarity(img_path):
    sim_map = self_similarity_map(img_path)
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
    plt.title("Original")
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(sim_map, cmap='inferno')
    plt.title("Self-Similarity Map")
    plt.colorbar(fraction=0.046)
    plt.show()

# Run on one forged example
row = train_df[train_df['has_mask']==1].sample(1, random_state=SEED).iloc[0]
visualize_self_similarity(row['image_path'])


In [ ]:
def visualize_keypoint_matches(img_path, max_matches=50):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # ORB keypoints
    orb = cv2.ORB_create(nfeatures=1000)
    kp, des = orb.detectAndCompute(gray, None)

    # Brute-force matcher (self-match within same image)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des, des)
    matches = sorted(matches, key=lambda x: x.distance)[:max_matches]

    match_img = cv2.drawMatches(img, kp, img, kp, matches, None,
                                matchColor=(0,255,0), singlePointColor=(255,0,0),
                                flags=2)
    plt.figure(figsize=(12,6))
    plt.imshow(cv2.cvtColor(match_img, cv2.COLOR_BGR2RGB))
    plt.title(f"Keypoint Match Map: {Path(img_path).stem}")
    plt.axis('off')
    plt.show()

# visualize one example
row = train_df[train_df['has_mask']==1].sample(1, random_state=SEED).iloc[0]
visualize_keypoint_matches(row['image_path'])


In [ ]:
# Average forged region coverage and region counts summary
forged_df = train_df[train_df['has_mask']==1]
print("Average forged pixel coverage:", forged_df["mask_coverage"].mean()*100, "%")
print("Median forged region count:", forged_df["num_regions"].median())
print("Average brightness:", forged_df["brightness"].mean())

plt.figure(figsize=(6,4))
sns.scatterplot(data=forged_df, x="brightness", y="mask_coverage", alpha=0.7)
plt.title("Brightness vs Forgery Coverage")
plt.show()
